In [1]:
import os

import sqlite3
import sqlalchemy
import pandas as pd

In [2]:
IPYTHON_HIST_FILE = os.path.join(os.environ['HOME'], '.ipython/profile_default/history.sqlite')

In [3]:
conn = sqlite3.connect(IPYTHON_HIST_FILE)
connal = sqlalchemy.create_engine('sqlite:///' + IPYTHON_HIST_FILE)

In [4]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,sessions
1,sqlite_sequence
2,history
3,output_history


In [5]:
columns = []
for tt in tables['name']:
    cur = conn.cursor()
    cur.execute("PRAGMA table_info({table_name:s})".format(table_name=tt))
    cols = pd.DataFrame(data=cur.fetchall(),
                        columns=[cc[0] for cc in cur.description])
    cols['table'] = tt
    columns += [cols]

columns = pd.concat(columns)
columns.set_index(['table', 'cid'], inplace=True)

In [6]:
# http://www.sqlite.org/pragma.html#pragma_table_info

columns

name       type  notnull dflt_value  pk
table           cid                                               
sessions        0       session    integer        0       None   1
                1         start  timestamp        0       None   0
                2           end  timestamp        0       None   0
                3      num_cmds    integer        0       None   0
                4        remark       text        0       None   0
sqlite_sequence 0          name                   0       None   0
                1           seq                   0       None   0
history         0       session    integer        0       None   1
                1          line    integer        0       None   2
                2        source       text        0       None   0
                3    source_raw       text        0       None   0
output_history  0       session    integer        0       None   1
                1          line    integer        0       None   2
                2        output       text        0       None   0

In [7]:
# `output_history` is empty

pd.read_sql('SELECT * FROM output_history;', conn)

,session,line,output
